In [1]:
import pandas as pd
import numpy as np
import requests, os
import http.client

# Importar Archivo de Excel desde Aduana.cl

In [2]:
http.client.HTTPConnection._http_vsn = 10
http.client.HTTPConnection._http_vsn_str = 'HTTP/1.0'

url="https://www.aduana.cl/aduana/site/docs/20181214/20181214093028/impo_pais_sa_monto_09062020.xlsx"

resp = requests.get(url)
with open('imports2020.xlsx', 'wb') as output:
    output.write(resp.content)

# Leer el archivo desde la ubicación de descarga

In [3]:
#Debe ingresar el directorio donde se descargo el archivo
imports = pd.read_excel(r'C:\Users\rey\Downloads\DAT256x\Module01\imports2020.xlsx')

# Quitar filas y columnas no deseadas

In [4]:
imports = imports.drop("Unnamed: 0",axis=1)
imports = imports.drop("Unnamed: 7",axis=1)
imports.columns = imports.iloc[2]
imports = imports.drop([0,1,2])

In [5]:
imports.head()

2,Pais / Código Arancelario,Enero - 2020,Febrero - 2020,Marzo - 2020,Abril -2020,Mayo -2020
3,Afghanistán,0,117.64,7105.42,0,0
4,39023000,0,0,7105.42,0,0
5,87149990,0,117.64,0,0,0
6,Albania,7922.41,19717.1,38627.8,2880,505.45
7,40169390,0,0,919.24,0,0


# Renombrar columnas de fechas

In [6]:
imports.rename(columns={'Enero - 2020':'Jan-2020',
                          'Febrero - 2020':'Feb-2020',
                          'Marzo - 2020':'Mar-2020',
                           "Abril -2020":"Apr-2020",
                           "Mayo -2020":"May-2020"},
                           
                 inplace=True)

In [7]:
imports.head()

2,Pais / Código Arancelario,Jan-2020,Feb-2020,Mar-2020,Apr-2020,May-2020
3,Afghanistán,0,117.64,7105.42,0,0
4,39023000,0,0,7105.42,0,0
5,87149990,0,117.64,0,0,0
6,Albania,7922.41,19717.1,38627.8,2880,505.45
7,40169390,0,0,919.24,0,0


# Hacer modificaciones al DataFrame

In [10]:
def convertir_serie_tiempo (df):
    
    #Esta función admite las columnas: ["Pais / Código Anrancelario", fecha en formato "mon-yr" en ingles
    
    #Crear la columna temporal "codigo_arancelario_temp" para retirar los espacios en el nombre de la columna "Pais / Código Arancelario" 
    
    df["codigo_arancelario_temp"]= df["Pais / Código Arancelario"]
    
    df = df.drop("Pais / Código Arancelario", axis=1)
          
    #Quitar los NaN de la columna "codigo_arancelario

    df = df.dropna(subset=["codigo_arancelario_temp"])
          
    #Crear la columna pais separando los valores con letras de los valores numericos y hacer el fill hacia abajo

    df["pais"] = df.codigo_arancelario_temp.str.extract('([a-zA-Z]+)', expand=True).ffill(axis=0)

    #Crear la columna codigo_arancelario y retirar los NaN

    df["codigo_arancelario"] = df.codigo_arancelario_temp.str.extract('([0-9]+)', expand=True)
    df =df.dropna(subset=["codigo_arancelario"])

    #Hacer Unpivot en columnas de tiempo

    df = df.drop(["codigo_arancelario_temp"], axis=1).melt(id_vars=['pais',"codigo_arancelario"], var_name='date', value_name='USD')

    #Quitar las filas con valor 0 en USD
     
    df = df[df.USD != 0]
    
    #Convertir date en datetime
    
    df["date"]=pd.to_datetime(df["date"])
    
    #Hacer la columna "date" en index
    
    df.set_index("date", inplace= True)
    
    return(df)

In [13]:
imports = convertir_serie_tiempo (imports)

In [14]:
imports

,pais,codigo_arancelario,USD
date,,,
2020-01-01,Albania,61062000,716.32
2020-01-01,Albania,62034900,167.9
2020-01-01,Albania,62044300,1540.32
2020-01-01,Albania,62046311,582.89
2020-01-01,Albania,62104000,4914.98
...,...,...,...
2020-05-01,Vietnam,96082020,315.58
2020-05-01,Vietnam,96099000,138.75
2020-05-01,Vietnam,96151100,2317.15
